In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [2]:
#cria dataframe
df=pd.read_csv('diamonds.csv')

In [3]:
#cria função para tratar data frame nas colunas categoricas e novas colunas com log
def create_log_columns(df,param):
    #cria colunas númericas para colunas categoricas com base na ordem de maior valor
    
    #na coluna clarity é atribuiudo valor de 1 a 8 conforme posição na lista
    cat_price=['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
    df['clarity_log'] = df['clarity'].apply(lambda x: 1+ (cat_price.index(x)))
    #na coluna color criado atribuido valor conforme ordem alfabetica
    color_list = sorted(list(df.color.unique()),reverse=True)
    df['color_log'] = df['color'].apply(lambda x: 1+ (color_list.index(x)))
    #na coluna cut dado valor conforme posição na lista
    cut_price = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
    df['cut_log'] = df['cut'].apply(lambda x: 1+ (cut_price.index(x)))
    
    #criação das colunas como log conforme os valores atuais
    #como na coluna rick não temos a coluna price, foi criado parametro que define se ela sera criada ou não
    if param == 1:
        df['price_log'] = np.log(df['price'])
    #para as demais foi feito o log sendo que nas colunas que possam ter zeros é feito log(x+1)
    df['carat_log']=np.log(df['carat'])
    df['cut_log']=np.log(df['cut_log'])
    df['color_log']=np.log(df['color_log'])
    df['clarity_log']=np.log(df['clarity_log'])
    df['depth_log']=np.log(df['depth'])
    df['table_log']=np.log(df['table'])
    df['x_log']=np.log(df['x']+1)
    df['y_log']=np.log(df['y']+1)
    df['z_log']=np.log(df['z']+1)
    return df

In [4]:
#aplica a função que trata o dataframe
df = create_log_columns(df,1)
#cria uma lista com todos dados unicos na coluna clarity
clar_lst = df.clarity.unique()
#cria uma lista com dataframes para cada tipo de clarity
dfs = [df.query(f'(clarity =="{x}")') for x in clar_lst]

In [8]:
def ger_model(df):
    #cria uma função que gera um modelo com base no dataframe inputa
    y= df['price_log']
    X= df[['carat_log','cut_log','color_log','depth_log','table_log','x_log','y_log','z_log']]
    modelo = LinearRegression()
    modelo.fit(X, y)
    return modelo

In [9]:
#para cada dataframe é gerado um modelo
modelos = [ger_model(df) for df in dfs]

In [10]:
def do_predict(row):
    #para cada linha é buscado o modelo correspondente e feito o predict
    #busca qual o clarity na linha
    clar = row['clarity']
    #define o input conforme os dados da linha
    X= [[row['carat_log'],row['cut_log'],row['color_log'],row['depth_log'],row['table_log'],row['x_log'],row['y_log'],row['z_log']]]
    #busca o modelo correspondente conforme a clarity
    index = list(clar_lst).index(clar)
    model = modelos[index] 
    #realiza a previsão
    y_predict = model.predict(X)
    #faz o exponencial para voltar a conversão feita no log
    y_predict=np.exp(y_predict)
    #retorna y_predict
    return y_predict
 

In [11]:
#cria um array com as previsões conforme cada linha do dataframe
y_predict = df.apply(lambda row : do_predict(row), axis=1)
#seleciona a coluna real para comparar com o previsto
y_real = df['price']
#calcula r2 do model
metrics.r2_score(y_real, y_predict).round(3)

0.966

In [12]:
#calcula rmse
np.sqrt(metrics.mean_squared_error(y_real,y_predict))

740.9133966289618

In [13]:
#busca o df do rick e faz uma cópia
rick_diamonds = pd.read_csv('rick_diamonds.csv')
rick_copy = rick_diamonds.copy()
#trata a copia para criar as colunas com log
rick_copy = create_log_columns(rick_copy,0)
#faz a previsão com a função criada para cada linha
y_predict = rick_copy.apply(lambda row : do_predict(row)[0], axis=1)
#cria uma coluna no dataframe principal com preco predito
rick_diamonds['price_predicted'] = y_predict
#salva o df
rick_diamonds.to_csv('./Deliverables/rick_diamonds_new.csv')